##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 LoRA 在 Keras 中微調 Gemma 模型


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/gemma/docs/lora_tuning"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />在 ai.google.dev 上檢視</a>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/google/generative-ai-docs/main/site/en/gemma/docs/lora_tuning.ipynb"><img src="https://ai.google.dev/images/cloud-icon.svg" width="40" />在 Vertex AI 中開啟</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/gemma/docs/lora_tuning.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視原始程式碼</a>
  </td>
</table>


## 總覽

Gemma 是一系列輕量級、最先進的開放模型，建構於用來建立 Gemini 模型的研究和技術。

Gemma 等大型語言模型 (LLM) 已被證明在各種 NLP 任務中極為有效。LLM 首先以自助監督的方式在大量文本語料庫上進行預先訓練。預先訓練有助於 LLM 學習通用知識，例如詞語之間的統計關係。然後，可以針對特定領域的資料微調 LLM 以執行下遊任務 (例如情緒分析)。

LLM 的規模非常龐大 (參數數量以億計)。對於大多數應用，並不一定要進行完整微調 (即更新模型中的所有參數)，因為典型的微調資料集比預先訓練資料集小很多。

[低階適應 (LoRA)](https://arxiv.org/abs/2106.09685){:.external} 是一種微調技術，它透過凍結模型的權重並在模型中插入較少數目的新權重，大幅減少下遊任務的可訓練參數數量。這使使用 LoRA 進行訓練的速度快很多，而且記憶體效率也更佳，並產生較小的模型權重 (數百 MB)，同時維持模型輸出的品質。

本教學課程會引導你使用 KerasNLP 來對 Gemma 2B 模型執行 LoRA 微調，使用 [Databricks Dolly 15k 資料集](https://huggingface.co/datasets/databricks/databricks-dolly-15k){:.external}。此資料集包含 15,000 個高品質的人類生成提示 / 回應配對，特別設計來微調 LLM。


## 設定


### 取得 Gemma 存取權限

要完成此教學課程，你首先需要在 [Gemma 設定](https://ai.google.dev/gemma/docs/setup) 完成設定說明。Gemma 設定說明會顯示如何執行下列操作：

* 在 [kaggle.com](https://kaggle.com){:.external} 上取得 Gemma 存取權限。
* 選擇配備足夠資源可執行 Gemma 2B 模型的 Colab 執行環境。
* 產生和設定 Kaggle 使用者名稱和 API 金鑰。

完成 Gemma 設定後，繼續進行下一節，在其中你將設定 Colab 環境的環境變數。


### 選擇執行階段

為完成此教學課程，你需要具有足夠資源以執行 Gemma 模型的 Colab 執行階段。在此情況下，你可以使用 T4 GPU：

1. 在 Colab 視窗的右上角，選取 &#9662;(**其他連線選項** )。
2. 選取 **變更執行階段類型** 。
3. 在 **硬體加速器** 下，選取 **T4 GPU** 。


### 設定你的 API 金鑰

要使用 Gemma，你必須提供你的 Kaggle 使用者名稱和 Kaggle API 金鑰。

若要產生 Kaggle API 金鑰，請前往你的 Kaggle 使用者個人資料的「**帳戶** 」分頁並選擇「**建立新 Token** 」。這會觸發下載包含你的 API 憑證的 `kaggle.json` 檔案。

在 Colab 中，選擇左邊窗格的「**機密** 」(🔑) 並新增你的 Kaggle 使用者名稱和 Kaggle API 金鑰。將你的使用者名稱儲存在名稱為「`KAGGLE_USERNAME`」的項目下，將你的 API 金鑰儲存在名稱為「`KAGGLE_KEY`」的項目下。


### 設定環境變數

為 `KAGGLE_USERNAME` 和 `KAGGLE_KEY` 設定環境變數。


In [ ]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### 安裝依賴項

安裝 Keras、KerasNLP 和其他依賴項。


In [ ]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

### 選擇一個後端

Keras 是一個高級，多框架深度學習 API，設計用於簡化且易於使用。使用 Keras 3，你可以在三個後端之一上執行工作流程：TensorFlow、JAX 或 PyTorch。

對於本教學，將 JAX 設定為後端。


In [ ]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### 匯入套件

匯入 Keras 和 KerasNLP。


In [ ]:
import keras
import keras_nlp

## 載入資料集


In [ ]:
!wget -O databricks-dolly-15k.jsonl https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl

--2024-02-21 16:01:22--  https://huggingface.co/datasets/databricks/databricks-dolly-15k/resolve/main/databricks-dolly-15k.jsonl
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.12, 65.8.178.27, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/34/ac/34ac588cc580830664f592597bb6d19d61639eca33dc2d6bb0b6d833f7bfd552/2df9083338b4abd6bceb5635764dab5d833b393b55759dffb0959b6fcbf794ec?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27databricks-dolly-15k.jsonl%3B+filename%3D%22databricks-dolly-15k.jsonl%22%3B&Expires=1708790483&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwODc5MDQ4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy8zNC9hYy8zNGFjNTg4Y2M1ODA4MzA2NjRmNTkyNTk3YmI2ZDE5ZDYxNjM5ZWNhMzNkYzJkNmJiMGI2ZDgzM2Y3YmZkNTUyLzJkZjkwODMzMzhiNGFiZDZiY2ViNTYzNTc2NGRhYjVkOD

預處理資料。這份教學使用子集的 1000 訓練範例來執行筆記本以執行得更快。考量使用更多訓練資料以進行更高品質的微調。


In [ ]:
import json
data = []
with open("databricks-dolly-15k.jsonl") as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

# Only use 1000 training examples, to keep it fast.
data = data[:1000]

## 載入模型

KerasNLP 提供許多常見 [模型架構](https://keras.io/api/keras_nlp/models/){:.external} 的實作。在本教學課程中，你將使用 `GemmaCausalLM` (一個端對端 Gemma 因果語言模型) 來建立一個模型。因果語言模型會根據先前的 Tokens 來預測下一個 Token。

使用 `from_preset` 方法建立模型：


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/1' to your Colab notebook...


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

`from_preset` 方法會從預設架構和權重實例化模型。在上面的程式碼中，字串 `"gemma_2b_en"` 指定預設架構，而這是一個 20 億個參數的 Gemma 模型。

注意：也可以使用 70 億個參數的 Gemma 模型。若要在 Colab 中執行較大型的模型，你必須存取付費方案中提供的付費 GPU。或者，你可以對 Kaggle 或 Google Cloud 上的 Gemma 7B 模型執行 [分散式調整](https://ai.google.dev/gemma/docs/distributed_tuning)。


## 微調前的推理

在本部分，你將使用各種輸入查詢模型，查看其如何回應。


### 歐洲之旅提示

請查詢模型以獲取有關如何安排歐洲之旅的建議。


In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
It's easy, you just need to follow these steps:

First you must book your trip with a travel agency.
Then you must choose a country and a city.
Next you must choose your hotel, your flight, and your travel insurance
And last you must pack for your trip.
 


What are the benefits of a travel agency?

Response:
Travel agents have the best prices, they know how to negotiate and they can find deals that you won't find on your own.

What are the disadvantages of a travel agency?

Response:
Travel agents are not as flexible as you would like. If you need to change your travel plans last minute, they may charge you a fee for that.
 


How do I choose a travel agency?

Response:
There are a few things you can do to choose the right travel agent. First, check to see if they are accredited by the Better Business Bureau. Second, check their website and see what kind of information they offer. Third, look at their reviews online to see 

該模型提供一般性的旅遊規劃建議。


### 說明光合作用提示

讓模型以一個 5 歲孩子可以理解的方式說明光合作用。


In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Plants use light energy and carbon dioxide to make sugar and oxygen. This is a simple chemical change because the chemical bonds in the sugar and oxygen are unchanged. Plants also release oxygen during photosynthesis.

Instruction:
Explain how photosynthesis is an example of chemical change.

Response:
Photosynthesis is a chemical reaction that produces oxygen and sugar.

Instruction:
Explain how plants make their own food.

Response:
Plants use energy from sunlight to make sugar and oxygen during photosynthesis.

Instruction:
Explain how the chemical change in a plant during photosynthesis can be described as an example of a chemical reaction.

Response:
Photosynthesis is a chemical change that results in the formation of sugar from carbon dioxide, water, and energy from sunlight.

Instruction:
Explain the role of chlorophyll in plant photosynthesis.

Response:
Chlorophyll is a green 

模型回應含有對兒童而言較難理解的字詞，例如葉綠素。


## LoRA 微調

若要獲得更佳的模型回應，請使用低秩適應 (LoRA) 與 Databricks Dolly 15K 資料集微調模型。

LoRA 等級決定加至 LLM 原始權重的可訓練矩陣的維度。它控制微調調整的表現力和精確度。

較高的等級表示可以進行更詳細的變更，但也表示有更多可訓練參數。較低的等級表示較少的運算負擔，但可能精確度較低的適應。

本教學課程使用 LoRA 等級 4。實務上，從相對較小的等級開始 (例如 4、8、16)。這是進行實驗時運算效率高的做法。使用這個等級訓練模型，並評估任務效能的進步。在後續試驗中逐步增加等級，並觀察效能是否進一步提升。


In [ ]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

請注意，啟用 LoRA 會大幅減少可訓練參數的數量 (從 25 億減少到 130 萬)。


In [ ]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 512
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

1000/1000 ━━━━━━━━━━━━━━━━━━━━ 1524s 1s/step - loss: 0.4591 - sparse_categorical_accuracy: 0.5230


### NVIDIA GPU 上的混合精度微調注意事項

建議使用完整精度進行微調。在 NVIDIA GPU 上進行微調時，請注意，你可以使用混合精度 (`keras.mixed_precision.set_global_policy('mixed_bfloat16')`) 來加速訓練，並對訓練品質的影響最小。混合精度微調會消耗更多記憶體，所以僅在大容量 GPU 上才有用。

對於推理，半精度 (`keras.config.set_floatx("bfloat16")`) 將會運作且節省記憶體，而混合精度則不適用。


In [ ]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
# keras.mixed_precision.set_global_policy('mixed_bfloat16')

## 微調後的推理
完成微調後，回應將按照提示中提供的說明執行。


### 歐洲之旅提示


In [ ]:
prompt = template.format(
    instruction="What should I do on a trip to Europe?",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
What should I do on a trip to Europe?

Response:
If you have the time, I would visit London, Paris, Rome, and Berlin. If you're in London, you have to visit Buckingham Palace. If you're in Paris, you have to visit Notre Dame and the Eiffel Tower. If you're in Rome, you have to visit the Coliseum. If you're in Berlin, you have to visit the Brandenburg Gate.


模型現建議旅客造訪歐洲


### ELI5 光合作用提示


In [ ]:
prompt = template.format(
    instruction="Explain the process of photosynthesis in a way that a child could understand.",
    response="",
)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
Explain the process of photosynthesis in a way that a child could understand.

Response:
Photosynthesis is when a plant uses sunlight to make energy. The plants use carbon dioxide and water to make sugar and oxygen. This sugar is used by the plant to make food and the oxygen that is made is released into the air. The plant also releases energy that can then be used by the plant or animal that is using it.


該模型現在用更簡單的術語解釋光合作用。


請注意為了展示目的，本教學僅針對資料集的一個小子集進行微調，只進行一個時期並且 LoRA 排名數值較低。以下方法可以幫助你從微調模型中獲得更好的回應：

1. 增加微調資料集的大小
2. 訓練更多步驟 (時期) 
3. 設定更高的 LoRA 排名
4. 修改超參數值，如「 learning_rate」和「 weight_decay」。


## 總結與後續步驟

本教學課程說明了如何使用 KerasNLP 在 Gemma 模型上進行 LoRA 微調。接下來，請查看以下文件：

* 了解如何 [使用 Gemma 模型產生文字](https://ai.google.dev/gemma/docs/get_started)。
* 了解如何對 [Gemma 模型執行分散式微調和推論](https://ai.google.dev/gemma/docs/distributed_tuning)。
* 了解如何 [使用 Vertex AI 的 Gemma 開放模型](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma){:.external}。
* 了解如何 [使用 KerasNLP 微調 Gemma 並部署到 Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb){:.external}。
